<a href="https://colab.research.google.com/github/hswarup/Exploratory-Analysis/blob/master/Spark_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Chicago's Reported Crime Data

## Download and install Spark

In [0]:
!ls

data   reported-crimes.csv	requirements.txt  spark-2.4.4-bin-hadoop2.7
drive  reported_crimes_parquet	sample_data	  spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,811

## Setup Spark environment

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp,col,lit
spark = SparkSession.builder.getOrCreate() 
spark

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Downloading and preprocessing Chicago's Reported Crime Data

In [0]:
#!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-04-15 19:00:29--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [  <=>               ]   1.56G  2.82MB/s    in 11m 11s 

2020-04-15 19:11:41 (2.38 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1674980963]



In [0]:
#!ls

 drive				 spark-2.4.4-bin-hadoop2.7
'rows.csv?accessType=DOWNLOAD'	 spark-2.4.4-bin-hadoop2.7.tgz
 sample_data			 spark-warehouse


##Rename the input file to a smaller/readable name

In [0]:
#!mv 'rows.csv?accessType=DOWNLOAD' reported-crimes.csv

##Read the input CSV File

In [0]:
#from pyspark.sql.functions import to_timestamp,col,lit
#rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
#rc.show(5)

## View the structure/schema of Input File

In [0]:
#rc.dtypes
#rc.printSchema()
#rc.columns

In [0]:
#rc.count()
#rc.describe().show()

## Create a Parquet File from .CSV File and store it in Google Drive
### This is done for better performance obtained from .parquet format(Compression and Using Parallel processing)

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')
# df2 = rc.select(
# 	col("ID"),\
# 	col("Case Number").alias("Case_Number"),\
# 	col("Date"),\
# 	col("Block"),\
# 	col("IUCR"),\
# 	col("Primary Type").alias("Primary_Type"),\
# 	col("Description"),\
# 	col("Location Description").alias("Location_Description"),\
# 	col("Arrest"),\
# 	col("Domestic"),\
# 	col("Beat"),\
# 	col("District"),\
# 	col("Ward"),\
# 	col("Community Area").alias("Community_Area"),\
# 	col("FBI Code").alias("FBICode"),\
# 	col("X Coordinate").alias("X_Coordinate"),\
# 	col("Y Coordinate").alias("Y_Coordinate"),\
# 	col("Year"),\
# 	col("Updated On").alias("Updated_On"),\
# 	col("Latitude"),\
# 	col("Longitude"),\
# 	col("Location")\
# )
#df2.show()
#df2.write.save("/content/drive/My Drive/data/reported_crimes_parquet",format="parquet")

## Read the input Parquet File

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!ls -lRt /content/drive/My\ Drive/data

rc = spark.read.format("parquet").load("/content/drive/My Drive/data/reported_crimes_parquet")
rc.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case_Number|               Date|               Block|IUCR|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|173

In [0]:
rc.count()

6752637

##A demo showing how to read a parquet file using pandas(takes a bit longer than spark)
*Also,The Colab instance crashed when running it for the 2nd time*



In [0]:
#import pandas as pd
#pandas_df=pd.read_parquet("/content/drive/My Drive/data/reported_crimes_parquet")
#pandas_df.sample(10)

In [0]:
#small_rc = rc.sample(fraction=0.0001)
#small_rc.show()

+-------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case_Number|               Date|               Block|IUCR|       Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|1730681|    G537925|2001-09-08 03:00:00|    054XX S DAMEN AV|041A|            BATTERY| AGGRAVATED: HANDGUN|              STREET|  true|

## Spark SQL (Create a Table)

In [0]:
rc.createOrReplaceTempView("reported_crimes")

In [0]:
spark.sql("SELECT COUNT(*) FROM reported_crimes ").show()

+--------+
|count(1)|
+--------+
| 6752637|
+--------+



##Resilient Distributed Datasets(Low Level)

In [0]:
#To read from .CSV
rdd = sc.textFile("reported-crimes.csv",)
rdd.first()
#type(rdd)
#help(sc.textFile)

# To read from parquet - but this is returning a dataframe instead of a RDD object and thus not correct
# from pyspark import SQLContext
# #help(dir(SQLContext))
# sqc = SQLContext(sc)
# sqc
# rdd = sqc.read.parquet("/content/drive/My Drive/data/reported_crimes_parquet")
# print (type(rdd))
# rdd.first()

Help on method textFile in module pyspark.context:

textFile(name, minPartitions=None, use_unicode=True) method of pyspark.context.SparkContext instance
    Read a text file from HDFS, a local file system (available on all
    nodes), or any Hadoop-supported file system URI, and return it as an
    RDD of Strings.
    
    If use_unicode is False, the strings will be kept as `str` (encoding
    as `utf-8`), which is faster and smaller than unicode. (Added in
    Spark 1.2)
    
    >>> path = os.path.join(tempdir, "sample-text.txt")
    >>> with open(path, "w") as testFile:
    ...    _ = testFile.write("Hello world!")
    >>> textFile = sc.textFile(path)
    >>> textFile.collect()
    ['Hello world!']



In [0]:
rdd_header = rdd.first()
rdd_header

Row(ID='1730080', Case_Number='G536683', Date=datetime.datetime(2001, 9, 6, 20, 0), Block='093XX S CRANDON AV', IUCR='0910', Primary_Type='MOTOR VEHICLE THEFT', Description='AUTOMOBILE', Location_Description='STREET', Arrest='true', Domestic='false', Beat='0413', District='004', Ward=None, Community_Area=None, FBICode='07', X_Coordinate='1193108', Y_Coordinate='1843138', Year='2001', Updated_On='08/17/2015 03:03:40 PM', Latitude='41.724529445', Longitude='-87.568237637', Location='(41.724529445, -87.568237637)')

In [0]:
rdd_new = rdd.filter(lambda line:line != rdd_header)
rdd_new.first()

TypeError: ignored

In [0]:
#rdd_new.filter(lambda line : line.split(",")).sample(withReplacement=False,fraction=0.0001).first()
output_rdd = rdd_new.filter(lambda line : line.split(",")[5]=='ASSAULT').\
                     map(lambda line : (line.split(",")[1],line.split(",")[6])).\
                     sample(withReplacement=False,fraction=0.0001).\
                     collect()
output_rdd

[('G648270', 'AGGRAVATED: HANDGUN'),
 ('HY270154', 'AGGRAVATED: HANDGUN'),
 ('HY394621', 'SIMPLE'),
 ('HY504731', 'AGGRAVATED: OTHER FIREARM'),
 ('HZ200806', 'SIMPLE'),
 ('HZ302671', 'SIMPLE'),
 ('HZ354501', 'SIMPLE'),
 ('HZ521821', 'AGGRAVATED: HANDGUN'),
 ('JB262796', 'AGGRAVATED: HANDGUN'),
 ('JB283580', 'SIMPLE'),
 ('JB395720', 'SIMPLE'),
 ('JB447394', 'SIMPLE'),
 ('JB570892', 'SIMPLE'),
 ('JC119602', 'SIMPLE'),
 ('JC208474', 'SIMPLE'),
 ('G340551', 'SIMPLE'),
 ('HH165841', 'AGGRAVATED: HANDGUN'),
 ('HH427720', 'SIMPLE'),
 ('HH541489', 'SIMPLE'),
 ('HH547902', 'AGGRAVATED:KNIFE/CUTTING INSTR'),
 ('HH707907', 'SIMPLE'),
 ('HJ206263', 'SIMPLE'),
 ('HJ680436', 'SIMPLE'),
 ('HK137699', 'AGGRAVATED: HANDGUN'),
 ('HK201784', 'SIMPLE'),
 ('HK647431', 'SIMPLE'),
 ('HK774664', 'SIMPLE'),
 ('HL409825', 'SIMPLE'),
 ('HL580434', 'SIMPLE'),
 ('HL658878', 'PRO EMP HANDS NO/MIN INJURY'),
 ('HL721934', 'SIMPLE'),
 ('HM650744', 'SIMPLE'),
 ('HN165367', 'SIMPLE'),
 ('HN208310', 'SIMPLE'),
 ('HN75938

##Display the first 5 rows of the column IUCR

In [0]:
# Spark SQL
spark.sql("SELECT iucr FROM reported_crimes LIMIT 5").show()

+----+
|iucr|
+----+
|0910|
|1710|
|0910|
|0820|
|0820|
+----+



In [0]:
# Dataframe API
rc.select(col("IUCR")).show(5)

+----+
|IUCR|
+----+
|0910|
|1710|
|0910|
|0820|
|0820|
+----+
only showing top 5 rows



##Display only the first 4 rows of the column names Case Number, Date and Arrest

In [0]:
#Spark SQL
spark.sql("SELECT case_number,date,arrest FROM reported_crimes LIMIT 4").show()

+-----------+-------------------+------+
|case_number|               date|arrest|
+-----------+-------------------+------+
|    G536683|2001-09-06 20:00:00|  true|
|    G534867|2001-09-06 20:12:38| false|
|    G536731|2001-09-02 23:30:00| false|
|    G532276|2001-09-05 17:15:00|  true|
+-----------+-------------------+------+



In [0]:
#Dataframe API
rc.select(["case_number","date","arrest"]).show(4)

+-----------+-------------------+------+
|case_number|               date|arrest|
+-----------+-------------------+------+
|    G536683|2001-09-06 20:00:00|  true|
|    G534867|2001-09-06 20:12:38| false|
|    G536731|2001-09-02 23:30:00| false|
|    G532276|2001-09-05 17:15:00|  true|
+-----------+-------------------+------+
only showing top 4 rows



## Add a column with name One, with entries all 1s 




In [0]:
# Spark SQL
add1_df = spark.sql("SELECT *,1 AS One FROM reported_crimes LIMIT 5").show()
add1_df

+-------+-----------+-------------------+------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|     ID|Case_Number|               Date|             Block|IUCR|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|One|
+-------+-----------+-------------------+------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|1730080|    G536683|2001-09-06 20:00:00|093XX S CRANDON AV|0910| MOTOR VEHICLE THEFT|          AUTOMOBILE|              STREET

In [0]:
# Dataframe API
add1_df = rc.withColumn("One",lit(1))
add1_df.show(5)

+-------+-----------+-------------------+------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|     ID|Case_Number|               Date|             Block|IUCR|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|One|
+-------+-----------+-------------------+------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|1730080|    G536683|2001-09-06 20:00:00|093XX S CRANDON AV|0910| MOTOR VEHICLE THEFT|          AUTOMOBILE|              STREET

##Remove the column IUCR

In [0]:
#Spark SQL
minus_iucr = spark.sql("SELECT  id,case_number,date,block,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbicode,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location FROM reported_crimes LIMIT 5")
minus_iucr.show()

+-------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     id|case_number|               date|             block|        primary_type|         description|location_description|arrest|domestic|beat|district|ward|community_area|fbicode|x_coordinate|y_coordinate|year|          updated_on|    latitude|    longitude|            location|
+-------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|1730080|    G536683|2001-09-06 20:00:00|093XX S CRANDON AV| MOTOR VEHICLE THEFT|          AUTOMOBILE|              STREET|  true|   false|0413|     004|n

In [0]:
#Dataframe API
rc.drop("IUCR").show(5)

+-------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case_Number|               Date|             Block|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|1730080|    G536683|2001-09-06 20:00:00|093XX S CRANDON AV| MOTOR VEHICLE THEFT|          AUTOMOBILE|              STREET|  true|   false|0413|     004|n

##Add the reported crimes for an additional day, 12-Nov-2018, to our dataset.

In [0]:
#Dataframe API (Tried using a read parquet..Was resulting in empty records..Hence using csv)
rc_12Nov2018 = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))
rc_12Nov2018.show(5)

#Before Union
#rc.count()
#6752637

#After Union
rc = rc.union(rc_12Nov2018)        ## Looks like the dataframe operations slowed down after this UNION operation
rc.count()
rc.orderBy("Date",ascending=False).show(5)


+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case_Number|               Date|               Block|IUCR|       Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00|  003XX S WHIPPLE ST|0810|              THEFT|           OVER $500|              STREET| fa

##What are the top 10 number of reported crimes by Primary type, in descending order of occurence?

In [0]:
#Spark SQL
spark.sql("SELECT COUNT(1) as reported_crime_count,primary_type FROM reported_crimes GROUP BY primary_type ORDER BY reported_crime_count DESC LIMIT 10").show()


+--------------------+-------------------+
|reported_crime_count|       primary_type|
+--------------------+-------------------+
|             1418421|              THEFT|
|             1232219|            BATTERY|
|              771498|    CRIMINAL DAMAGE|
|              711647|          NARCOTICS|
|              418832|      OTHER OFFENSE|
|              418507|            ASSAULT|
|              388028|           BURGLARY|
|              314141|MOTOR VEHICLE THEFT|
|              265517| DECEPTIVE PRACTICE|
|              255599|            ROBBERY|
+--------------------+-------------------+



In [0]:
# Dataframe API
rc.groupBy("primary_type").count().orderBy("count",ascending=False).select(["count","primary_type"]).show(10)

+-------+-------------------+
|  count|       primary_type|
+-------+-------------------+
|1418421|              THEFT|
|1232219|            BATTERY|
| 771498|    CRIMINAL DAMAGE|
| 711647|          NARCOTICS|
| 418832|      OTHER OFFENSE|
| 418507|            ASSAULT|
| 388028|           BURGLARY|
| 314141|MOTOR VEHICLE THEFT|
| 265517| DECEPTIVE PRACTICE|
| 255599|            ROBBERY|
+-------+-------------------+
only showing top 10 rows



##What percentage of reported crimes resulted in an arrest?

In [0]:
#Dataframe API
rc.filter(col("arrest") == "true").count()*100/rc.count()

27.75412035327828

In [0]:
#Spark SQL
spark.sql("SELECT (COUNT(1) * 100 / (SELECT COUNT(1) FROM reported_crimes)) as Arrest_percent FROM  reported_crimes WHERE arrest = 'true'").show()

+-----------------+
|   Arrest_percent|
+-----------------+
|27.75412035327828|
+-----------------+



## What are the top 3 locations for reported crimes?

In [0]:
#Dataframe API
rc.groupBy("location_description").count().orderBy("count",ascending=False).show(3)

+--------------------+-------+
|location_description|  count|
+--------------------+-------+
|              STREET|1770573|
|           RESIDENCE|1144625|
|           APARTMENT| 698154|
+--------------------+-------+
only showing top 3 rows



In [0]:
#Spark SQL
spark.sql("SELECT COUNT(1),location_description FROM reported_crimes GROUP BY location_description ORDER BY COUNT(1) DESC LIMIT 3").show()

+--------+--------------------+
|count(1)|location_description|
+--------+--------------------+
| 1770573|              STREET|
| 1144625|           RESIDENCE|
|  698154|           APARTMENT|
+--------+--------------------+



##What is 3 days earlier that the oldest date and 3 days later than the most recent date?

In [0]:
#Dataframe API
from pyspark.sql import functions
from pyspark.sql.functions import min,max,date_add,date_sub

rc.select(date_sub(min(col("Date")),3).alias("min_date"),date_add(max(col("Date")),3).alias("max_date")).show()

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2000-12-29|2018-11-13|
+----------+----------+



In [0]:
#Spark SQL
spark.sql("SELECT date_sub(min(date),3) min_date,date_add(max(date),3) max_date FROM reported_crimes").show()

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2000-12-29|2018-11-13|
+----------+----------+



## Create a Dataframe from scratch and convert string to date,timestamp

In [0]:
from pyspark.sql.functions import to_date,to_timestamp
df = spark.createDataFrame([("2019-12-25 01:30:00 PM","ABC")],["Christmas_Day","random_col"])
df.show(truncate=False)

#Dataframe API
df.select(to_date("Christmas_Day","yyyy-MM-dd hh:mm:ss aa"),to_timestamp("Christmas_Day","yyyy-MM-dd hh:mm:ss aa")).show()

#Spark SQL
df.createOrReplaceTempView("date_table")
spark.sql("SELECT to_date(Christmas_Day,'yyyy-MM-dd hh:mm:ss aa'),to_timestamp(Christmas_Day,'yyyy-MM-dd hh:mm:ss aa') FROM date_table").show()

+----------------------+----------+
|Christmas_Day         |random_col|
+----------------------+----------+
|2019-12-25 01:30:00 PM|ABC       |
+----------------------+----------+

+--------------------------------------------------+-------------------------------------------------------+
|to_date(`Christmas_Day`, 'yyyy-MM-dd hh:mm:ss aa')|to_timestamp(`Christmas_Day`, 'yyyy-MM-dd hh:mm:ss aa')|
+--------------------------------------------------+-------------------------------------------------------+
|                                        2019-12-25|                                    2019-12-25 13:30:00|
+--------------------------------------------------+-------------------------------------------------------+

+-------------------------------------------------------------+------------------------------------------------------------------+
|to_date(date_table.`Christmas_Day`, 'yyyy-MM-dd hh:mm:ss aa')|to_timestamp(date_table.`Christmas_Day`, 'yyyy-MM-dd hh:mm:ss aa')|
+----------

##Working with JOINs

In [0]:
!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
!ls -lt
!head police-station.csv
ps = spark.read.csv('police-station.csv',header=True).withColumnRenamed("DISTRICT NAME","DISTRICT_NAME")
ps.show()
ps.dtypes

--2020-04-15 20:54:15--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-station.csv’

police-station.csv      [ <=>                ]   5.57K  --.-KB/s    in 0s      

2020-04-15 20:54:15 (513 MB/s) - ‘police-station.csv’ saved [5699]

total 2085176
drwxr-xr-x  2 root root       4096 Apr 15 18:45  spark-warehouse
drwx------  4 root root       4096 Apr 15 18:44  drive
-rw-r--r--  1 root root 1674980963 Apr 15 11:10  reported-crimes.csv
drwxr-xr-x  1 root root       4096 Apr  3 16:24  sample_data
-rw-r--r--  1 root root  230091034 Aug 27  2019  spark-2.4.4-bin-hadoop2.7.tgz
-rw-r--r--  1 root root  230091034 Aug 27  2019  spark-2.4.4-bin-hadoop2.7.tgz.1
drwxr-xr-x 

[('DISTRICT', 'string'),
 ('DISTRICT_NAME', 'string'),
 ('ADDRESS', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('WEBSITE', 'string'),
 ('PHONE', 'string'),
 ('FAX', 'string'),
 ('TTY', 'string'),
 ('X COORDINATE', 'string'),
 ('Y COORDINATE', 'string'),
 ('LATITUDE', 'string'),
 ('LONGITUDE', 'string'),
 ('LOCATION', 'string')]

##The reported crimes dataset has only the district number. Add the district name by joining with the police station dataset

In [0]:
from pyspark.sql.functions import lpad
#Spark SQL
ps.createOrReplaceTempView("police_station")
spark.sql("SELECT rc.*,ps.DISTRICT_NAME FROM reported_crimes rc LEFT OUTER JOIN police_station ps ON rc.district = LPAD(ps.district,3,0)").show()


+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------+
|     ID|Case_Number|               Date|               Block|IUCR|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|    Latitude|    Longitude|            Location| DISTRICT_NAME|
+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------+
|1730080|    G536683|2001-09-06 20:00:00|  093XX S CRANDON AV|0910| MOTOR VEHICLE THEFT|

In [0]:
#Dataframe API
rc.cache()
ps = ps.withColumn("format_district",lpad(col("district"),3,"0"))
#ps.columns
#rc.dtypes
rc.join(ps,ps.format_district == rc.District,"left_outer").drop('ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION',
 'format_district').show()


+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+--------+--------------+
|     ID|Case_Number|               Date|               Block|IUCR|        Primary_Type|         Description|Location_Description|Arrest|Domestic|Beat|District|Ward|Community_Area|FBICode|X_Coordinate|Y_Coordinate|Year|          Updated_On|DISTRICT| DISTRICT_NAME|
+-------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+-------+------------+------------+----+--------------------+--------+--------------+
|1730080|    G536683|2001-09-06 20:00:00|  093XX S CRANDON AV|0910| MOTOR VEHICLE THEFT|          AUTOMOBILE|              STREET|  true|   false|0413|     004|null|          null|     07|     1193108|    

##What is the most frequently reported non-criminal activity

In [0]:
#Spark SQL
spark.sql("SELECT count(1) num,description FROM reported_crimes WHERE primary_type LIKE 'NON-CRIMINAL%' GROUP BY description ORDER BY num DESC LIMIT 3").show()

+---+--------------------+
|num|         description|
+---+--------------------+
|107|       LOST PASSPORT|
| 37|   FOID - REVOCATION|
|  9|NOTIFICATION OF C...|
+---+--------------------+



In [0]:
#Dataframe API
rc.cache()
rc.select(col("primary_type")).distinct().show(100,truncate=False)
#rc.show()
#rc.filter((col("primary_type") == "NON-CRIMINAL")  | (col("primary_type") == "NON-CRIMINAL ")).show()
rc.filter((col("primary_type") == "NON-CRIMINAL")  | (col("primary_type") == "NON-CRIMINAL (SUBJECT SPECIFIED)")).groupBy(col("Description")).count().orderBy("count",ascending=False).show()


+--------------------+-----+
|         Description|count|
+--------------------+-----+
|       LOST PASSPORT|  107|
|   FOID - REVOCATION|   37|
|NOTIFICATION OF C...|    9|
|NOTIFICATION OF S...|    8|
|CONCEALED CARRY L...|    5|
|      FOUND PASSPORT|    4|
|GUN OFFENDER NOTI...|    3|
+--------------------+-----+



##Which day of the week has the most number of reported crime

In [0]:
#Spark SQL
spark.sql("SELECT COUNT(1) num,SUBSTR(date,9,2) FROM reported_crimes GROUP BY SUBSTR(date,9,2) ORDER BY num DESC LIMIT 10").show()

+------+-------------------------------------+
|   num|substring(CAST(date AS STRING), 9, 2)|
+------+-------------------------------------+
|266599|                                   01|
|229215|                                   15|
|225363|                                   20|
|223812|                                   10|
|222567|                                   17|
|222217|                                   14|
|221511|                                   16|
|221220|                                   12|
|221044|                                   18|
|220596|                                   09|
+------+-------------------------------------+



In [0]:
#Dataframe API
from pyspark.sql.functions import substring,dayofweek,date_format

#dateofthemonth
rc.groupBy(substring(col("Date"),9,2).alias("Day")).count().orderBy("count",ascending=False).show()
#dateoftheweek
rc.groupBy(dayofweek(col("Date"))).count().orderBy("count",ascending=False).show()
#dayoftheweek
rc.groupBy(date_format(col("Date"),"E")).count().orderBy("count",ascending=False).show()


+---+------+
|Day| count|
+---+------+
| 01|266599|
| 15|229215|
| 20|225363|
| 10|223812|
| 17|222567|
| 14|222217|
| 16|221511|
| 12|221220|
| 18|221044|
| 09|220596|
| 21|220433|
| 05|220386|
| 19|220289|
| 06|220116|
| 07|220078|
| 08|220050|
| 23|219668|
| 11|219531|
| 28|219481|
| 13|219442|
+---+------+
only showing top 20 rows

+---------------+-------+
|dayofweek(Date)|  count|
+---------------+-------+
|              6|1016872|
|              4| 973785|
|              3| 967957|
|              7| 965084|
|              5| 964447|
|              2| 952637|
|              1| 911855|
+---------------+-------+

+--------------------+-------+
|date_format(Date, E)|  count|
+--------------------+-------+
|                 Fri|1016872|
|                 Wed| 973785|
|                 Tue| 967957|
|                 Sat| 965084|
|                 Thu| 964447|
|                 Mon| 952637|
|                 Sun| 911855|
+--------------------+-------+



## Define custom schema and datatypes when reading data from a CSV

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,BooleanType,DoubleType
rc.printSchema()
custom_schema = StructType([StructField("ID",IntegerType(),True),\
                            StructField("Case_num",StringType(),True) \
                            ])
print (custom_schema)
ps_test = spark.read.csv("police-station.csv",header=True,schema=custom_schema).show()

root
 |-- ID: string (nullable = true)
 |-- Case_Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary_Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location_Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community_Area: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- X_Coordinate: string (nullable = true)
 |-- Y_Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated_On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)

StructType(List(StructField(ID,IntegerType,true),StructField(Case_num,StringType,true)))
+----+---------

##RDD - How many police stations are there?

In [0]:
# Initialize the RDD by copying the data from another Dataframe
#ps_rdd = ps.rdd.map(lambda line : (line[0],line[1]))
#ps_rdd1 = ps.rdd.map(lambda line : line)
#ps_rdd.take(3)
#ps_rdd1.take(3)

#or
#Initialize it from a file contents
ps_rdd = sc.textFile("police-station.csv")
ps_rdd_header = ps_rdd.first()

ps_rdd = ps_rdd.filter(lambda line:line != ps_rdd_header)
ps_rdd.take(3)
# type(ps_rdd)
#ps_rdd.map(lambda line:line.split(",")).count()

#ps_rdd_header
#ps_rdd.top(5)

['1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/districts/1st-district-central/,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.85837259,-87.62735617,"(41.8583725929, -87.627356171)"',
 '6,Gresham,7808 S Halsted St,Chicago,IL,60620,http://home.chicagopolice.org/community/districts/6th-district-gresham/,312-745-3617,312-745-3649,312-745-3639,1172283.013,1853022.646,41.75213684,-87.64422891,"(41.7521368378, -87.6442289066)"',
 '11,Harrison,3151 W Harrison St,Chicago,IL,60612,http://home.chicagopolice.org/community/districts/11th-district-harrison/,312-746-8386,312-746-4281,312-746-5151,1155244.069,1897148.755,41.87358229,-87.70548813,"(41.8735822883, -87.705488126)"']

## Display the District ID, District name, Address and Zip for the police station with District ID 7


In [0]:
# RDD
ps_rdd.filter(lambda line : line.split(",")[0] == "7").map(lambda line : (line.split(",")[0],line.split(",")[1],line.split(",")[2],line.split(",")[5])).collect()


# Dataframe API
ps.filter(col("district") == "7").select(col("district"),col("district_name"),col("address"),col("zip")).show()

#Spark SQL
spark.sql("SELECT district,district_name,address,zip FROM police_station WHERE district = 7").show()

['1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/districts/1st-district-central/,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.85837259,-87.62735617,"(41.8583725929, -87.627356171)"',
 '6,Gresham,7808 S Halsted St,Chicago,IL,60620,http://home.chicagopolice.org/community/districts/6th-district-gresham/,312-745-3617,312-745-3649,312-745-3639,1172283.013,1853022.646,41.75213684,-87.64422891,"(41.7521368378, -87.6442289066)"',
 '11,Harrison,3151 W Harrison St,Chicago,IL,60612,http://home.chicagopolice.org/community/districts/11th-district-harrison/,312-746-8386,312-746-4281,312-746-5151,1155244.069,1897148.755,41.87358229,-87.70548813,"(41.8735822883, -87.705488126)"']

##RDD - Police stations 10 and 11 are geographically close to each other. Display the District ID, District name, address and zip code

In [0]:
ps_rdd.filter(lambda line : line.split(",")[0] == "10" or line.split(",")[0] =="11").map(lambda line : (line.split(",")[0],line.split(",")[1],line.split(",")[2],line.split(",")[5])).take(3)

[('11', 'Harrison', '3151 W Harrison St', '60612'),
 ('10', 'Ogden', '3315 W Ogden Ave', '60623')]